In [1]:
import numpy as np

In [2]:
def f(x):
    x1=x[0]
    x2=x[1]
    r1=100*(x2-(x1**2))**2
    r2=(1-x1)**2
    result=r1+r2
    return result

def gradf(x):
    x1=x[0]
    x2=x[1]
    g1=(-400*x1*(x2-(x1**2)))-2*(1-x1)
    g2=200*(x2-(x1**2))
    g=np.array([g1,g2])
    return g
    
def hessian(x):
    x1=x[0]
    x2=x[1]
    result_1=[-400*x2+1200*(x1**2)+2, -400*x1]
    result_2=[-400*x1,200]
    result=np.array([result_1,result_2])
    return result

In [3]:
def inverse_of_regularised_hessian(A,epsilon = 0.01):
    try:
        Ainv = np.linalg.inv(A)
    except:
        # find inverse of A + epsilon * I
        I = np.ones(A.shape)
        Ainv = np.linalg.inv(A + epsilon * I)
    return Ainv
    

def find_best_step_size(f,gradf,x,d,alpha,beta,sigma):
    found_alpha = False
    while not found_alpha:
        if f(x + alpha * d) <= f(x) + sigma * alpha * np.dot(gradf(x),d):
            found_alpha = True
        else:
            alpha = beta * alpha
    return alpha

def gradient_descent_newton(f,x0,gradf,hessian,
                            convergence_thresh=0.12,
                            beta=0.8,
                            sigma=0.6,
                            max_iters=10**5,
                            print_progress=True):
    """
    @f is the function to be minimized
    @gradf is the gradient of the function to be minimized
    @x0 is the initial guess
    @convergence_thresh is the threshold used to determine convergence
    @max_iters is the maximum number of iteration we try before giving up
    @print_progress flag to indicate whether to print the progress of the algorithm 
    @beta, @sigma are parameters to the optimizer algorithm that finds best step size.
    """
    converged = False
    num_iters_so_far = 0
    x = x0
    trajectory = []
    alpha = 1 # initial guess of step size
    while not converged:
        # find descent direction
        d = - gradf(x) / np.linalg.norm(gradf(x))
        
        # choose step size
        alpha = find_best_step_size(f,gradf,x,d,alpha,beta,sigma)
        
        
        # compute regularized hessian
        B = inverse_of_regularised_hessian(hessian(x))
        
        # update x
        num_iters_so_far += 1
        trajectory.append(x)
        x = x + alpha * B@d
        
        # check convergence
        grad_f_norm = np.linalg.norm(gradf(x))
        if grad_f_norm <= convergence_thresh:
            converged = True 
            trajectory.append(x)
            print("\nConverged! The minimial solution occurs @x = ", x, "\nwith value f(x)=",f(x), "\nand ||gradf||=",grad_f_norm)
        if not converged and num_iters_so_far > max_iters:
            converged = True
            print("Failed to converge :(")
            
        # output progress
        if print_progress and num_iters_so_far<=10:
            print("\nIteration: ", num_iters_so_far,"\nx: ",x, " f(x)", f(x), "\n||gradf||:",grad_f_norm) 
            print("prev_x: ", trajectory[-1], " f(prev_x): ",f(trajectory[-1]))
            print("B: ",B)
            print("--------------------------------------------------------")
    return x,np.array(trajectory) 

x = np.array([0.5,-0.5])
x_opt,trajectory = gradient_descent_newton(f,x,gradf,hessian)



Iteration:  1 
x:  [ 0.50000513 -0.49883248]  f(x) 56.3257712008902 
||gradf||: 211.09842835012444
prev_x:  [ 0.5 -0.5]  f(prev_x):  56.5
B:  [[0.00331126 0.00331126]
 [0.00331126 0.00831126]]
--------------------------------------------------------

Iteration:  2 
x:  [ 0.50001028 -0.49766494]  f(x) 56.15181261561272 
||gradf||: 210.77074667082246
prev_x:  [ 0.50000513 -0.49883248]  f(prev_x):  56.3257712008902
B:  [[0.00331636 0.0033164 ]
 [0.0033164  0.00831643]]
--------------------------------------------------------

Iteration:  3 
x:  [ 0.50001544 -0.49649738]  f(x) 55.978124244094715 
||gradf||: 210.44306498635294
prev_x:  [ 0.50001028 -0.49766494]  f(prev_x):  56.15181261561272
B:  [[0.00332149 0.00332155]
 [0.00332155 0.00832162]]
--------------------------------------------------------

Iteration:  4 
x:  [ 0.50002062 -0.49532981]  f(x) 55.80470608626287 
||gradf||: 210.11538329669182
prev_x:  [ 0.50001544 -0.49649738]  f(prev_x):  55.978124244094715
B:  [[0.00332662 0.0033